## Command Line


This is the most fundamental way to deploy Dask on multiple machines. In production environments, this process is often automated by some other resource manager. Hence, it is rare that people need to follow these instructions explicitly. But since we want learn how to "build" a cluster we want study how to start it from the command line.

A ```dask.distributed``` network consists of one ```dask-scheduler``` process and several ```dask-worker``` processes that connect to that scheduler. These are normal Python processes that can be executed from the command line. We launch the dask-scheduler executable in one process and the dask-worker executable in several processes, possibly on different machines.

Hence, respect to the revious lectures, today we want to create a cluster "from scratch" using IP addresses and real workers and not the automatic "local cluster" created by dask.



At first let's see how works the ```dask-scheduler``` command

In [1]:
%%bash

dask-scheduler --help

Usage: dask-scheduler [OPTIONS] [PRELOAD_ARGV]...

Options:
  --host TEXT                   URI, IP or hostname of this server
  --port INTEGER                Serving port
  --interface TEXT              Preferred network interface like 'eth0' or
                                'ib0'
  --protocol TEXT               Protocol like tcp, tls, or ucx
  --tls-ca-file PATH            CA cert(s) file for TLS (in PEM format)
  --tls-cert PATH               certificate file for TLS (in PEM format)
  --tls-key PATH                private key file for TLS (in PEM format)
  --bokeh-port INTEGER          Deprecated.  See --dashboard-address
  --dashboard-address TEXT      Address on which to listen for diagnostics
                                dashboard  [default: :8787]
  --dashboard / --no-dashboard  Launch the Dashboard [default: --dashboard]
  --bokeh / --no-bokeh          Deprecated.  See --dashboard/--no-dashboard.
  --show / --no-show            Show web UI [default: --show]
  --dashboard-p

In [2]:
%%bash 

#dask-scheduler run it inside a real bash terminal not from jupyter

The command ```dask-worker``` on the rest of the nodes. Let's see how it works:

In [3]:
%%bash

dask-worker --help

Usage: dask-worker [OPTIONS] [SCHEDULER] [PRELOAD_ARGV]...

Options:
  --tls-ca-file PATH              CA cert(s) file for TLS (in PEM format)
  --tls-cert PATH                 certificate file for TLS (in PEM format)
  --tls-key PATH                  private key file for TLS (in PEM format)
  --worker-port TEXT              Serving computation port, defaults to
                                  random. When creating multiple workers with
                                  --nprocs, a sequential range of worker ports
                                  may be used by specifying the first and last
                                  available ports like <first-port>:<last-
                                  port>. For example, --worker-port=3000:3026
                                  will use ports 3000, 3001, ..., 3025, 3026.
  --nanny-port TEXT               Serving nanny port, defaults to random. When
                                  creating multiple nannies with --nprocs, a
            

Thisn command must be run by providing the address to the node that hosts ```dask-scheduler```:


In [4]:
%%bash
#dask-worker 192.168.1.12:8687 run int inside the real command line not from jupyter

### Basic concepts
The scheduler and workers both need to accept TCP connections on an open port. By default, the scheduler binds to port 8786 and the worker binds to a random open port. If you are behind a firewall then you may have to open particular ports or tell Dask on a different port.

Dask workers are run within a *nanny* process that *monitors* the worker process and restarts it if necessary.

As we have saw last lecture, Dask schedulers and even workers host interactive diagnostic web servers using the Bokeh server. These are optional, but generally useful to users. The diagnostic server on the scheduler is particularly valuable, and is served on port 8787.

### Try to create a your first cluster

At first run the cell below in order to indentify your network-card and what your IP is:

In [7]:
%%bash 

ifconfig ##only for those of you that does not use the docker cluster

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 333318  bytes 135236769 (135.2 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 333318  bytes 135236769 (135.2 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

wlp1s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 10.4.51.1  netmask 255.255.255.252  broadcast 10.4.51.3
        inet6 fe80::4be3:10e8:6a10:e891  prefixlen 64  scopeid 0x20<link>
        ether 48:f1:7f:76:de:82  txqueuelen 1000  (Ethernet)
        RX packets 649635  bytes 932589943 (932.5 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 67685  bytes 7244369 (7.2 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



Now open you command line and run:

In [ ]:
#dask-scheduler --host IP --port 8786

At this point open another command line and run this command:

In [ ]:
#dask-worker IP:8786 --nprocs 2 #this command create 2 workers

when is all up to date try to connect to the dashboard and take a look to your new cluster!
If you have install all the packages in the correct way you should be able to access to the dashboard at: IP:8787

### Run an old exercise over the new cluster

Try to count how many words are present in all the documents over the cluster.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from dask.distributed import Client
import time

categories = [
     'comp.graphics',
     'comp.os.ms-windows.misc',
     'comp.sys.ibm.pc.hardware',
     'comp.sys.mac.hardware',
     'comp.windows.x',
     'misc.forsale',
     'rec.autos',
     'rec.motorcycles',
     'rec.sport.baseball',
     'rec.sport.hockey',
     'sci.crypt',
     'sci.electronics',
     'sci.med',
     'sci.space'
]

dataset = fetch_20newsgroups(subset='train', categories=categories ).data

print("Texts document present on the dataset: "+str(len(dataset)))

def count_word_in_statement(text):
    """
    This function takes a text as input and return the number of the words that it contains
    """
    #time.sleep(0.1)
    splitted_words = text.split()
    return len(splitted_words)

Texts document present on the dataset: 8283


Sequential code:

In [2]:
import time
start = time.time()


total_words_in_all_data = 0
for index in range(0, len(dataset)):
    total_words_in_all_data = total_words_in_all_data + count_word_in_statement(dataset[index])

    
end = time.time()
print("Total word in the dataset: {}".format(total_words_in_all_data))
print("Computation took {}s".format(end-start))

Total word in the dataset: 2038444
Computation took 0.10924959182739258s


Distributed code:

In [2]:
client = Client() #change your setting

In [12]:
import time
start = time.time()

#futures = client.map(count_word_in_statement, dataset)
futures = [client.submit(count_word_in_statement, data) for data in dataset]

futures = client.submit(sum, futures)
total_words_in_all_data = client.gather(futures)

    
end = time.time()
print("Total word in the dataset: {}".format(total_words_in_all_data))
print("Computation took {}s".format(end-start))


Total word in the dataset: 2038444
Computation took 8.460427522659302s


In [4]:
client.close()

### Why in this case sequential code took more than 10 times les than ditributed version?

In general when yuo have to deal with a cluster you have to think about the *overhead*. 

You can image the overhead like the the computational time necessary to process your data. 
Typically in a cluster there two kind of overhead:
+ scheduler overhead in serializing the objects that must be sent to workers
+ connection overhead. The speed of the network connection between the cluster nodes

In the first case, the scheduler adds about one millisecond of overhead per task or Future object. Despite this may sound fast or inconsequential, it's quite slow if you run a large number of tasks. Under this perspective, a larger number of the task means a larger amount of time to create the Future objects of the tasks. 
In the light of above, if your functions run faster than 100ms or so then you might not see any
speedup from using distributed computing, but even worse, probably you might see that the performances get worse.

In the second case things are different. The connection overhead may depends from several factors including the stability of the network, the type (wired or WiFi or optic fibe), and bandwith of the network.

This is what is happening in the previous example.

Let's try to introduce a simulation of intesive computation (a sleep of 10ms: 10 times less the the overhead generated by dask-scheduler):

In [14]:
def count_word_in_statement(text):
    """
    This function takes a text as input and return the number of the words that it contains
    """
    splitted_words = text.split()
    time.sleep(0.01)
    return len(splitted_words)

Try to run and wait the sequantial code:

In [15]:
import time
start = time.time()


total_words_in_all_data = 0
for index in range(0, len(dataset)):
    total_words_in_all_data = total_words_in_all_data + count_word_in_statement(dataset[index])
    end = time.time() - start
    if end >= 50:
        print("More than {}s of computation time...".format(end))

    
end = time.time()
print("Total word in the dataset: {}".format(total_words_in_all_data))
print("Computation took {}s".format(end-start))

More than 50.00693130493164s of computation time...
More than 50.01743984222412s of computation time...
More than 50.02785897254944s of computation time...
More than 50.038312673568726s of computation time...
More than 50.048792362213135s of computation time...
More than 50.0592885017395s of computation time...
More than 50.06975722312927s of computation time...
More than 50.08012270927429s of computation time...
More than 50.090423345565796s of computation time...
More than 50.1008186340332s of computation time...
More than 50.11118769645691s of computation time...
More than 50.12155818939209s of computation time...
More than 50.13215613365173s of computation time...
More than 50.142539501190186s of computation time...
More than 50.15362548828125s of computation time...
More than 50.16425824165344s of computation time...
More than 50.1750214099884s of computation time...
More than 50.185537338256836s of computation time...
More than 50.1962833404541s of computation time...
More than 5

More than 51.67832136154175s of computation time...
More than 51.688660621643066s of computation time...
More than 51.69953989982605s of computation time...
More than 51.70994687080383s of computation time...
More than 51.72026300430298s of computation time...
More than 51.73065805435181s of computation time...
More than 51.74120020866394s of computation time...
More than 51.75148558616638s of computation time...
More than 51.761849880218506s of computation time...
More than 51.772300720214844s of computation time...
More than 51.782801151275635s of computation time...
More than 51.793100357055664s of computation time...
More than 51.803548097610474s of computation time...
More than 51.81402349472046s of computation time...
More than 51.82440376281738s of computation time...
More than 51.834795236587524s of computation time...
More than 51.845295906066895s of computation time...
More than 51.85593628883362s of computation time...
More than 51.866432189941406s of computation time...
Mor

More than 53.34764838218689s of computation time...
More than 53.35813856124878s of computation time...
More than 53.36851119995117s of computation time...
More than 53.378875494003296s of computation time...
More than 53.38925766944885s of computation time...
More than 53.39965605735779s of computation time...
More than 53.40996026992798s of computation time...
More than 53.42028069496155s of computation time...
More than 53.430726766586304s of computation time...
More than 53.44148254394531s of computation time...
More than 53.45193076133728s of computation time...
More than 53.46226477622986s of computation time...
More than 53.47256803512573s of computation time...
More than 53.4829306602478s of computation time...
More than 53.49328589439392s of computation time...
More than 53.50372672080994s of computation time...
More than 53.514206647872925s of computation time...
More than 53.524691343307495s of computation time...
More than 53.53504776954651s of computation time...
More than

More than 55.0188570022583s of computation time...
More than 55.02930760383606s of computation time...
More than 55.03997802734375s of computation time...
More than 55.0504937171936s of computation time...
More than 55.06087565422058s of computation time...
More than 55.07133603096008s of computation time...
More than 55.0817928314209s of computation time...
More than 55.09220504760742s of computation time...
More than 55.10249733924866s of computation time...
More than 55.112993478775024s of computation time...
More than 55.12336826324463s of computation time...
More than 55.133721351623535s of computation time...
More than 55.14414167404175s of computation time...
More than 55.15477418899536s of computation time...
More than 55.16533946990967s of computation time...
More than 55.175724029541016s of computation time...
More than 55.18610382080078s of computation time...
More than 55.1966769695282s of computation time...
More than 55.2072114944458s of computation time...
More than 55.2

More than 56.6858606338501s of computation time...
More than 56.69930815696716s of computation time...
More than 56.70960712432861s of computation time...
More than 56.72006177902222s of computation time...
More than 56.73064398765564s of computation time...
More than 56.741037130355835s of computation time...
More than 56.75139331817627s of computation time...
More than 56.7619423866272s of computation time...
More than 56.7723491191864s of computation time...
More than 56.78266739845276s of computation time...
More than 56.79302477836609s of computation time...
More than 56.803380250930786s of computation time...
More than 56.813732862472534s of computation time...
More than 56.824142932891846s of computation time...
More than 56.834503412246704s of computation time...
More than 56.844871044158936s of computation time...
More than 56.85521984100342s of computation time...
More than 56.86557459831238s of computation time...
More than 56.875956296920776s of computation time...
More tha

...

More than 77.14421248435974s of computation time...
More than 77.15479230880737s of computation time...
More than 77.1654098033905s of computation time...
More than 77.17593431472778s of computation time...
More than 77.18641376495361s of computation time...
More than 77.19696307182312s of computation time...
More than 77.20764088630676s of computation time...
More than 77.2185127735138s of computation time...
More than 77.23017120361328s of computation time...
More than 77.24068021774292s of computation time...
More than 77.25124549865723s of computation time...
More than 77.26173496246338s of computation time...
More than 77.27226257324219s of computation time...
More than 77.28280901908875s of computation time...
More than 77.29335856437683s of computation time...
More than 77.30387234687805s of computation time...
More than 77.3144142627716s of computation time...
More than 77.32503628730774s of computation time...
More than 77.3357675075531s of computation time...
More than 77.346

More than 78.82711744308472s of computation time...
More than 78.83756709098816s of computation time...
More than 78.8480372428894s of computation time...
More than 78.8584566116333s of computation time...
More than 78.87070918083191s of computation time...
More than 78.88118886947632s of computation time...
More than 78.89169478416443s of computation time...
More than 78.90242171287537s of computation time...
More than 78.91311693191528s of computation time...
More than 78.92361903190613s of computation time...
More than 78.93425178527832s of computation time...
More than 78.94495344161987s of computation time...
More than 78.95539140701294s of computation time...
More than 78.96589612960815s of computation time...
More than 78.97671794891357s of computation time...
More than 78.98717999458313s of computation time...
More than 78.9976065158844s of computation time...
More than 79.00809049606323s of computation time...
More than 79.01853799819946s of computation time...
More than 79.02

More than 80.69502639770508s of computation time...
More than 80.70529699325562s of computation time...
More than 80.71560215950012s of computation time...
More than 80.72621417045593s of computation time...
More than 80.73648858070374s of computation time...
More than 80.74673819541931s of computation time...
More than 80.75696969032288s of computation time...
More than 80.76719665527344s of computation time...
More than 80.77744698524475s of computation time...
More than 80.78776788711548s of computation time...
More than 80.79820156097412s of computation time...
More than 80.80871152877808s of computation time...
More than 80.81922149658203s of computation time...
More than 80.82964587211609s of computation time...
More than 80.8401267528534s of computation time...
More than 80.85057640075684s of computation time...
More than 80.8613064289093s of computation time...
More than 80.87187242507935s of computation time...
More than 80.8823721408844s of computation time...
More than 80.89

More than 82.37295842170715s of computation time...
More than 82.38468837738037s of computation time...
More than 82.39536714553833s of computation time...
More than 82.4059534072876s of computation time...
More than 82.41677641868591s of computation time...
More than 82.42712831497192s of computation time...
More than 82.43767285346985s of computation time...
More than 82.44813060760498s of computation time...
More than 82.45849442481995s of computation time...
More than 82.4689564704895s of computation time...
More than 82.47941946983337s of computation time...
More than 82.4899070262909s of computation time...
More than 82.50081777572632s of computation time...
More than 82.51154732704163s of computation time...
More than 82.52190971374512s of computation time...
More than 82.53246760368347s of computation time...
More than 82.54296255111694s of computation time...
More than 82.55343079566956s of computation time...
More than 82.56392049789429s of computation time...
More than 82.57

More than 84.25341010093689s of computation time...
More than 84.2639844417572s of computation time...
More than 84.27436184883118s of computation time...
More than 84.28478860855103s of computation time...
More than 84.2953052520752s of computation time...
More than 84.30573582649231s of computation time...
More than 84.31625533103943s of computation time...
More than 84.32690739631653s of computation time...
More than 84.3374285697937s of computation time...
More than 84.34822845458984s of computation time...
More than 84.35881233215332s of computation time...
More than 84.3692798614502s of computation time...
More than 84.37962746620178s of computation time...
More than 84.39008140563965s of computation time...
More than 84.40071725845337s of computation time...
More than 84.41117095947266s of computation time...
More than 84.4216639995575s of computation time...
More than 84.43252515792847s of computation time...
More than 84.44365811347961s of computation time...
More than 84.4541

Distributed version:

In [24]:
client = Client() #change your setting

/home/lorenzo/.local/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40613 instead
  warnings.warn(


In [25]:
import time
start = time.time()

futures = [client.submit(count_word_in_statement, data) for data in dataset]
futures = client.submit(sum, futures)
total_words_in_all_data = client.gather(futures)

    
end = time.time()
print("Total word in the dataset: {}".format(total_words_in_all_data))
print("Computation took {}s".format(end-start))

Total word in the dataset: 2038444
Computation took 13.156180381774902s


In [4]:
client.close()

What happens if we increment the number of workers or threads per worker?

If you feel like a hero and you don't be afraid to become old by standing in front of the PC, you can try to compare the sequential code and the distributed code by increasing the sleep time 100ms or 1s

### How works the distribution and the scheduling of the processes?

#### How a worker is choosen?
Even though you can reduce and make some restrictions, e.g: restriction over worker resources, Dask automatically decides the suitable workers for your tasks by figuring out the optimized worker for each task.
This means that, if a task has significant data dependencies or if the workers are under heavy load then this choice of worker can strongly impact global performance because the decision becomes heavy.

Dask follows the following rules before to assign a task to a worker:
+ If the task has no major dependencies and no restrictions then we find the least occupied worker.

+ if a task has user-provided restrictions (for example it must run on a machine with a GPU) then we restrict the available pool of workers to just that set, otherwise, we consider all workers
+ from the pool of workers Dask determinates the workers to whom the least amount of data would need to be transferred (means less overhead on the cluster and hence computation optimization).
+  if some dependencies in the graph can be broken the will be assigned to the worker that currently has the fewest tasks.


Dask also allows modifying the worker decision function in order to be more flexible and to improve the customization of a cluster. This means that particular processes or particular computational fields in which performances can be improved by customizing and optimizing the task's assignation decision can be made more performant.

Breaking the dependencies in some cases is necessary, especially if each node has a lot of sons. In this case, each node with his sons must be removed from the graph and computed alone. This has a huge impact on performances and memory. On the other hand, this means that when a user submits a task, the computation graph must be scan to figuring out and optimizing this kind of dependencies.


#### How choose the next task?
Typically Dask follows those rules in order to choose the next task that must be executed:
+ Run tasks on a first-come-first-served basis for fairness between multiple clients
+ Run tasks that are part of the critical path in an effort to reduce total running time and minimize straggler workloads
+ Run tasks that allow us to release many dependencies in an effort to keep the memory footprint small
+ Run tasks that are related so that large chunks of work can be completely eliminated before running new chunks of work

As you can see a part of the overhead on a cluster is principally caused by the optimization of the execution task decision. Even though these are rules implemented by Dask, in general, the majority of the Cluster, even if they are based on other frameworks and other architectures, follow the same similar approaches.

On the other hand, some computational fields may require a different approach to decide which tasks can be executed, e.g: by using last-in-first-out approach or by giving a priority to each task in order to execute first some processes.

In some cases, Dask optimization exploit also a partially last-in-first-out approach. When a worker finishes a task the immediate dependencies of that task get top priority. This encourages a behavior of finishing ongoing work immediately before starting new work. This often conflicts with the first-come-first-served objective but often results in shorter total runtimes and significantly reduced memory footprints.


#### Where these decisions are made?

The decision are basically made ina small steps and in a different computation steps by client, scheduler, and workers:

+ As we submit a graph from the *client* to the scheduler we automatically assign a numeric priority to each task of that graph. This priority focuses on computing deeply before broadly, preferring critical paths, and preferring nodes with many dependencies.

+ When the graph reaches the scheduler the scheduler changes each of these numeric priorities into a tuple of two numbers, the first of which is an increasing counter, the second of which is the client-generated priority described above. This per-graph counter encourages a first-in-first-out policy between computations. All tasks from a previous call to compute have a higher priority than all tasks from a subsequent call to compute (or submit, persist, map, or any operation that generates futures).

+ Whenever a task is ready to run the scheduler assigns it to a worker. The scheduler does not wait based on priority. However when the worker receives these tasks it considers their priorities when determining which tasks to prioritize for communication or for computation. 


### Worker Resources

Let's suppose that you want to run a proces over a cluster, but only in those machine that has a GPU or have at least 16Gb of RAM. Now let's imagin that you have a cluster of ten computers in which four have a GPU while the others no. In this case we want to balance tasks across the cluster with these resource constraints in mind, allocating GPU-constrained tasks to GPU-enabled workers. Additionally we need to be sure to constrain the number of GPU tasks that run concurrently on any given worker to ensure that we respect the provided limits.
Clearly, this situation arises not only for GPUs but for many resources like tasks that require a large amount of memory at runtime, special disk access, or access to special hardware.

When you require workers with particular resources you must be sure that those resources are availables over the cluster.
Otherwise your processes should be never executed.

Let's try an example togheter: 

At first, stop the workers that you have in your cluster and the scheduler. Start again the scheduler and then turn up two workers with those commands:

+ ```dask-worker dask-scheduler:8786 --nprocs 1 --nthreads 1 --resources "GPU=2"```
+ ```dask-worker dask-scheduler:8786 --nprocs 1 --nthreads 1 --resources "GPU=1"```

In [3]:
import numpy as np
from dask.distributed import Client

client = Client() ##change your settings

/home/lorenzo/.local/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39077 instead
  warnings.warn(


In [4]:
client

Client Scheduler: tcp://127.0.0.1:42877 Dashboard: http://127.0.0.1:39077/status,Cluster Workers: 4 Cores: 8 Memory: 7.56 GiB


In [5]:
matrices = []

for i in range(100):
    matrices.append(np.random.rand(4,3))

def compute_polynomial_kernel(matrix):
    polynomial_degree = 2
    return np.power((np.dot(matrix, matrix.T)+1), 2)

Since we are working on a matrix computation let's suppose that we want exploit the multi-gpu available only on some workers. 
Assume that we need to use 3 GPUs:

In [ ]:
processed = [client.submit(compute_polynomial_kernel, matrix, resources={'GPU': 3}) for matrix in matrices]

kernels = client.gather(processed)

for i in kernels:
    print("Kernel is: {}".format(i))
    print()
client.close()

In [7]:
client.close()

nothing is happening but the code still running...... Let's try to add on-the-fly a worker with 3 GPUs

```dask-worker 192.168.1.12:8786 --nprocs 1 --nthreads 1 --resources "GPU=3"```

### Exercise 1:

Compute the traces of all the generated matrix. Execute this code over 2 workers with 2 "SpecialCPU" each one.
You must use the ```map``` function.

In [4]:
import numpy as np
matrices = [np.random.randint(low=m, high=m+1, size=(4, 3)) for m in (range(11))]

In [11]:
#dask-worker scheduler:8786 --nprocs 2 --resources "GPU=2" command to be run for the process to start

processed = [client.submit(np.trace, matrix, resources={'GPU': 2}) for matrix in matrices]

print(client.gather(processed))

cmap = client.map(np.trace, matrices, resources = {'GPU':2})

print(client.gather(cmap))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30]


### Exercise 2:

Execute the code of the "howmany_within_range" exercise from the previous lecture, into a worker with 256Gb of RAM. Map function is not allowed.
    

    

In [23]:
import numpy as np

## creation of a matrix of 100 rows and 10 columns with each value between 0 and 100.
np.random.RandomState(42)
arr = np.random.randint(0, 100, size=[100, 10])

from dask.distributed import Client
client = Client()
client

/home/lorenzo/.local/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38231 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:34135 Dashboard: http://127.0.0.1:38231/status,Cluster Workers: 4 Cores: 8 Memory: 7.56 GiB


In [24]:
def howmany_within_range(row):
    """Returns how many numbers lie within 0 and 10 in the given `row`"""
    count = 0
    for n in row:
        if 0 <= n <= 10:
            count = count + 1
    return count

#necessary command for cell execution
#dask-worker  scheduler:port --nprocs 2 --resources "MEMORY=256e9"

futures = [client.submit(howmany_within_range, matrix, resources={'MEMORY': 256e9}) for matrix in arr]

results = client.gather(futures)
print(results)

client.close()

[0, 0, 1, 1, 2, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 2, 0, 2, 2, 1, 1, 0, 3, 1, 2, 1, 3, 1, 2, 2, 1, 2, 1, 1, 0, 0, 3, 0, 1, 0, 2, 1, 1, 1, 1, 0, 0, 1, 2, 2, 0, 1, 4, 2, 0, 4, 0, 2, 1, 0, 1, 1, 4, 0, 0, 1, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 0, 1, 1, 2, 2, 0, 0, 0, 1, 2, 2, 1, 1, 1]
